In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install open3d

In [ ]:
import torch.utils.data as data
import os
import glob
import PIL.Image as Image
import numpy as np
import torch
import open3d as o3d
import matplotlib.pyplot as plt


class AI4MarsDataset(data.Dataset):
    """
    A dataset class for the AI4Mars MSL data. It includes the grayscale images, depth data, and semantic labels.
    Parameters:
    -----------
    folder_path: the path to the MSL directory of the AI4Mars dataset
    is_train: True/False whether the dataset should be for the training or testing data
    
    """
    def __init__(self, folder_path, is_train, image_size=256):
        super(AI4MarsDataset, self).__init__()
        self.image_size = image_size
        self.img_files = []
        self.depth_files = []
        self.label_files = []
        if is_train:
            all_label_files = glob.glob(os.path.join(folder_path,'labels','train','*.png'))
        else:
            all_label_files = glob.glob(os.path.join(folder_path,'labels','test','masked-gold-min1-100agree','*.png'))
        for label_path in all_label_files:
            if is_train: 
                base_name = os.path.basename(label_path)[:-4]
            else:
                base_name = os.path.basename(label_path)[:-11]
            depth_path = os.path.join(folder_path,'images','rng_256',base_name[0:13]+'RNG'+base_name[16:]+".tiff")
            image_path = os.path.join(folder_path,'images','edr',base_name+".JPG")
            if os.path.exists(depth_path) and os.path.exists(image_path):
                self.img_files.append(image_path)
                self.depth_files.append(depth_path)
                self.label_files.append(label_path)

    def __getitem__(self, index):
        image_size = self.image_size
        img_path = self.img_files[index]
        label_path = self.label_files[index]
        depth_path = self.depth_files[index]
        image = Image.open(img_path)
        depth = Image.open(depth_path)
        label = Image.open(label_path)
        image = image.resize((image_size,image_size),Image.ANTIALIAS)
        depth = depth.resize((image_size,image_size),Image.ANTIALIAS)
        label = label.resize((image_size,image_size),Image.NEAREST)
        image = np.asarray(image)
        depth = np.asarray(depth)
        label = np.asarray(label)
        image = torch.from_numpy(image).float()
        depth = torch.from_numpy(depth).float()
        label = torch.from_numpy(label).long()
        
        return image, depth, label

    def __len__(self):
        return len(self.img_files)

    def display_image(self, index):
        image_size = self.image_size
        img_path = self.img_files[index]
        depth_path = self.depth_files[index]
        image = Image.open(img_path)
        depth = Image.open(depth_path)
        image = image.resize((image_size,image_size),Image.ANTIALIAS)
        depth = depth.resize((image_size,image_size),Image.ANTIALIAS)
        image = np.ascontiguousarray(image, dtype=np.float32).reshape(image_size,image_size,1)
        depth = np.ascontiguousarray(depth, dtype=np.float32).reshape(image_size,image_size,1)
        color_raw = o3d.geometry.Image(image)
        depth_raw = o3d.geometry.Image(depth)
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw)
        plt.subplot(1, 2, 1)
        plt.title('Grayscale Image')
        plt.imshow(rgbd_image.color)
        plt.subplot(1, 2, 2)
        plt.title('Depth Image')
        plt.imshow(rgbd_image.depth)
        plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
     
    def forward(self, x):
        return self.conv_block(x)

class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.maxpool = nn.MaxPool2d(2)
        self.conv_block = ConvBlock(in_channels, out_channels)
    
    def forward(self, x):
        x1 = self.maxpool(x)
        x2 = self.conv_block(x1)

        return x2

class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
            super().__init__()

            self.upscale = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv_block = ConvBlock(in_channels, out_channels)

    
    def forward(self, x, enc_x):
        x1 = self.upscale(x)

        delta_Y = enc_x.size()[2] - x1.size()[2]
        delta_X = enc_x.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [delta_X // 2, delta_X - delta_X // 2,
                        delta_Y // 2, delta_Y - delta_Y // 2])
        
        x1 = torch.cat([enc_x,x1], dim=1)

        x2 = self.conv_block(x1)

        return x2 
    
class UNet(nn.Module):
    def __init__(self, in_channels, n_classes):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.n_classes = n_classes

        
        self.enc0= ConvBlock(in_channels, 64)
        self.enc1 = Encoder(64, 128)
        self.enc2 = Encoder(128, 256)
        self.enc3 = Encoder(256, 512)
        self.enc4 = Encoder(512, 1024)
        self.dec1 = Decoder(1024, 512)
        self.dec2 = Decoder(512, 256)
        self.dec3 = Decoder(256, 128)
        self.dec4 = Decoder(128, 64)
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.enc0(x)
        x2 = self.enc1(x1)
        x3 = self.enc2(x2)
        x4 = self.enc3(x3)
        x5 = self.enc4(x4)
        x = self.dec1(x5, x4)
        x = self.dec2(x, x3)
        x = self.dec3(x, x2)
        x = self.dec4(x, x1)
        logits = self.outc(x)
        return logits


In [ ]:
from zipfile import ZipFile
import pandas as pd
import os, time
import shutil
import sys
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow import keras
import random

In [ ]:
pip install torchmetrics

In [ ]:
import torchmetrics

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pickle
from torch.autograd import Variable

In [ ]:
pip install Pillow==9.0.0

In [ ]:
def main(num_epochs=5, batch_size=2, dataroot="/content/drive/MyDrive/data/data_subset/msl", image_size = 256):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device == torch.device('cpu'):
        num_workers = 0
    else:
        torch.cuda.empty_cache()
        num_workers = 2
    # load and transform dataset

    train_dataset = AI4MarsDataset(folder_path=dataroot, is_train=True, image_size=image_size)
    test_dataset = AI4MarsDataset(folder_path=dataroot, is_train=False, image_size=image_size)

    train_size = int(256/256 * len(train_dataset))
    eval_size = len(train_dataset) - train_size
    train_dataset, eval_dataset = torch.utils.data.random_split(train_dataset, [train_size, eval_size])
    
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                              shuffle=True, num_workers=num_workers)
    evalloader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size,
                                              shuffle=False, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                             shuffle=False, num_workers=num_workers)

    model = UNet(in_channels=1,n_classes=1)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())

    model.to(device)
    best_model_test_acc = 0
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        
        running_loss = 0.0
        train_loss, test_loss = 0.0, 0.0
        train_acc, test_acc = 0.0, 0.0
        train_total, test_total = 0, 0
        train_correct, test_correct = 0, 0
        model.train()
        
        for data in trainloader:
            # get the inputs; data is a list of [inputs, labels]
            images, depths, labels = data
            num_in_batch = images.shape[0]
            images=images.reshape(num_in_batch,1,image_size,image_size)
            depths=depths.reshape(num_in_batch,1,image_size,image_size)
            labels = labels.reshape(num_in_batch,image_size,image_size).long()
            labels[labels>=255]=4
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(images.to(device))
            train_loss = criterion(outputs, depths.to(device))
            train_loss.backward()
            optimizer.step()

            # print statistics
            train_loss += train_loss.item()
        with torch.no_grad():
          model.eval()
          for data in testloader:
                images, depths, labels = data
                num_in_batch = images.shape[0]
                images=images.reshape(num_in_batch,1,image_size,image_size)
                depths=depths.reshape(num_in_batch,1,image_size,image_size)
                labels = labels.reshape(num_in_batch,image_size,image_size).long()
                labels[labels==255]=4
                outputs = model(images.to(device))
                test_loss = criterion(outputs, depths.to(device))
                test_loss += test_loss.item()
        print('Epoch %d| Train loss: %.4f|Test loss: %.4f|'%(
            epoch+1, train_loss/len(trainloader), test_loss/len(testloader)))
        
        torch.save(model.state_dict(), './model.pth')
          
    print('Finished Training')


if __name__ == "__main__":
    main()

Epoch 1| Train loss: 2.5758|Test loss: 4.2758|
Epoch 2| Train loss: 1.2660|Test loss: 3.8343|
Epoch 3| Train loss: 1.0599|Test loss: 3.9559|
Epoch 4| Train loss: 0.6707|Test loss: 3.4649|
Epoch 5| Train loss: 0.5434|Test loss: 2.3009|
Finished Training


In [ ]:
def main(num_epochs=5, batch_size=2, dataroot="/content/drive/MyDrive/data/data_subset/msl", image_size = 256):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device == torch.device('cpu'):
        num_workers = 0
    else:
        torch.cuda.empty_cache()
        num_workers = 2
    # load and transform dataset

    train_dataset = AI4MarsDataset(folder_path=dataroot, is_train=True, image_size=image_size)
    test_dataset = AI4MarsDataset(folder_path=dataroot, is_train=False, image_size=image_size)

    train_size = int(256/256 * len(train_dataset))
    eval_size = len(train_dataset) - train_size
    train_dataset, eval_dataset = torch.utils.data.random_split(train_dataset, [train_size, eval_size])
    
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                              shuffle=True, num_workers=num_workers)
    evalloader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size,
                                              shuffle=False, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                             shuffle=False, num_workers=num_workers)

    model = UNet(in_channels=1,n_classes=1)
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters())

    model.to(device)
    best_model_test_acc = 0
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        
        running_loss = 0.0
        train_loss, test_loss = 0.0, 0.0
        train_acc, test_acc = 0.0, 0.0
        train_total, test_total = 0, 0
        train_correct, test_correct = 0, 0
        model.train()
        
        for data in trainloader:
            # get the inputs; data is a list of [inputs, labels]
            images, depths, labels = data
            num_in_batch = images.shape[0]
            images=images.reshape(num_in_batch,1,image_size,image_size)
            depths=depths.reshape(num_in_batch,1,image_size,image_size)
            labels = labels.reshape(num_in_batch,image_size,image_size).long()
            labels[labels>=255]=4
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(images.to(device))
            train_loss = criterion(outputs, depths.to(device))
            train_loss.backward()
            optimizer.step()

            # print statistics
            train_loss += train_loss.item()
        with torch.no_grad():
          model.eval()
          for data in testloader:
                images, depths, labels = data
                num_in_batch = images.shape[0]
                images=images.reshape(num_in_batch,1,image_size,image_size)
                depths=depths.reshape(num_in_batch,1,image_size,image_size)
                labels = labels.reshape(num_in_batch,image_size,image_size).long()
                labels[labels==255]=4
                outputs = model(images.to(device))
                test_loss = criterion(outputs, depths.to(device))
                test_loss += test_loss.item()
        print('Epoch %d| Train loss: %.4f|Test loss: %.4f|'%(
            epoch+1, train_loss/len(trainloader), test_loss/len(testloader)))
        
        torch.save(model.state_dict(), './model.pth')
          
    print('Finished Training')


if __name__ == "__main__":
    main()

Epoch 1| Train loss: 0.0467|Test loss: 0.1014|
Epoch 2| Train loss: 0.1125|Test loss: 0.0795|
Epoch 3| Train loss: 0.0921|Test loss: 0.1187|
Epoch 4| Train loss: 0.0614|Test loss: 0.0891|
Epoch 5| Train loss: 0.0490|Test loss: 0.0698|
Finished Training


In [ ]:
class RMSE(nn.Module):
    def _init_(self):
        super(RMSE, self)._init_()

    def forward(self, preds, targets):

        rmse = torch.sqrt(torch.mean(torch.pow(targets - preds, 2)))

        return rmse

In [ ]:
def main(num_epochs=5, batch_size=2, dataroot="/content/drive/MyDrive/data/data_subset/msl", image_size = 256):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device == torch.device('cpu'):
        num_workers = 0
    else:
        torch.cuda.empty_cache()
        num_workers = 2
    # load and transform dataset

    train_dataset = AI4MarsDataset(folder_path=dataroot, is_train=True, image_size=image_size)
    test_dataset = AI4MarsDataset(folder_path=dataroot, is_train=False, image_size=image_size)

    train_size = int(256/256 * len(train_dataset))
    eval_size = len(train_dataset) - train_size
    train_dataset, eval_dataset = torch.utils.data.random_split(train_dataset, [train_size, eval_size])
    
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                              shuffle=True, num_workers=num_workers)
    evalloader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size,
                                              shuffle=False, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                             shuffle=False, num_workers=num_workers)

    model = UNet(in_channels=1,n_classes=1)
    criterion = RMSE()
    optimizer = torch.optim.Adam(model.parameters())

    model.to(device)
    best_model_test_acc = 0
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        
        running_loss = 0.0
        train_loss, test_loss = 0.0, 0.0
        train_acc, test_acc = 0.0, 0.0
        train_total, test_total = 0, 0
        train_correct, test_correct = 0, 0
        model.train()
        
        for data in trainloader:
            # get the inputs; data is a list of [inputs, labels]
            images, depths, labels = data
            num_in_batch = images.shape[0]
            images=images.reshape(num_in_batch,1,image_size,image_size)
            depths=depths.reshape(num_in_batch,1,image_size,image_size)
            labels = labels.reshape(num_in_batch,image_size,image_size).long()
            labels[labels>=255]=4
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(images.to(device))
            train_loss = criterion(depths.to(device),outputs)
            train_loss.backward()
            optimizer.step()

            # print statistics
            train_loss += train_loss.item()
        with torch.no_grad():
          model.eval()
          for data in testloader:
                images, depths, labels = data
                num_in_batch = images.shape[0]
                images=images.reshape(num_in_batch,1,image_size,image_size)
                depths=depths.reshape(num_in_batch,1,image_size,image_size)
                labels = labels.reshape(num_in_batch,image_size,image_size).long()
                labels[labels==255]=4
                outputs = model(images.to(device))
                test_loss = criterion(depths.to(device),outputs)
                test_loss += test_loss.item()
        print('Epoch %d| Train loss: %.4f|Test loss: %.4f|'%(
            epoch+1, train_loss/len(trainloader), test_loss/len(testloader)))
        
        torch.save(model.state_dict(), './model.pth')
          
    print('Finished Training')


if __name__ == "__main__":
    main()

Epoch 1| Train loss: 0.0473|Test loss: 0.2363|
Epoch 2| Train loss: 0.0899|Test loss: 0.2082|
Epoch 3| Train loss: 0.1666|Test loss: 0.1886|
Epoch 4| Train loss: 0.1079|Test loss: 0.1788|
Epoch 5| Train loss: 0.0978|Test loss: 0.1876|
Finished Training
